In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("parkinson_5%.csv")

In [3]:
df

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.90,NaN,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0,1.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,NaN,3.18,7.19,...,234.51,170.0,169.0,0.01,NaN,2.25,0.0,0.00,23.0,1.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0,1.0
3,2.50,0.0,1.16,1.63,3.47,13.51,NaN,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,NaN,0.0,0.00,23.0,1.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0.0
1036,1.76,NaN,NaN,0.69,2.43,7.57,0.74,2.72,NaN,11.93,...,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0,0.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,NaN,14.24,...,149.64,79.0,77.0,0.01,NaN,20.00,1.0,4.57,1.0,0.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,451.37,75.0,70.0,0.01,0.0,NaN,3.0,19.58,1.0,0.0


In [4]:
df.dtypes

Jitter (local)                         float64
Jitter (local, absolute)               float64
Jitter (rap)                           float64
Jitter (ppq5)                          float64
Jitter (ddp)                           float64
Shimmer (local)                        float64
Shimmer (local, dB)                    float64
Shimmer (apq3)                         float64
Shimmer (apq5)                         float64
Shimmer (apq11)                        float64
Shimmer (dda)                          float64
AC                                     float64
NTH                                    float64
HTN                                    float64
Median pitch                           float64
Mean pitch                             float64
Standard deviation                     float64
Minimum pitch                          float64
Maximum pitch                          float64
Number of pulses                       float64
Number of periods                      float64
Mean period  

In [5]:
df.isnull().sum()

Jitter (local)                         56
Jitter (local, absolute)               55
Jitter (rap)                           60
Jitter (ppq5)                          52
Jitter (ddp)                           59
Shimmer (local)                        37
Shimmer (local, dB)                    54
Shimmer (apq3)                         44
Shimmer (apq5)                         71
Shimmer (apq11)                        60
Shimmer (dda)                          51
AC                                     50
NTH                                    51
HTN                                    59
Median pitch                           54
Mean pitch                             69
Standard deviation                     52
Minimum pitch                          53
Maximum pitch                          52
Number of pulses                       57
Number of periods                      46
Mean period                            56
Standard deviation of period           53
Fraction of locally unvoiced frame

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 29120
nilai hilang : 1456
persentase : 5.0


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Jitter (local)                         5.384615
Jitter (local, absolute)               5.288462
Jitter (rap)                           5.769231
Jitter (ppq5)                          5.000000
Jitter (ddp)                           5.673077
Shimmer (local)                        3.557692
Shimmer (local, dB)                    5.192308
Shimmer (apq3)                         4.230769
Shimmer (apq5)                         6.826923
Shimmer (apq11)                        5.769231
Shimmer (dda)                          4.903846
AC                                     4.807692
NTH                                    4.903846
HTN                                    5.673077
Median pitch                           5.192308
Mean pitch                             6.634615
Standard deviation                     5.000000
Minimum pitch                          5.096154
Maximum pitch                          5.000000
Number of pulses                       5.480769
Number of periods                      4

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,7)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    
    if tertimbang == 0:
        tertimbang = dummy.iloc[index-1, col]
        
    dummy.iloc[index,col] = tertimbang
    print("row, col :", index, col)
    print(" ")

In [10]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [11]:
data = df.iloc[:, :-1]
df2 = df.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.90,0.00,2.70,8.33,0.78,4.52,4.61,6.80,...,187.58,160.0,159.0,0.01,0.0,0.00,0.0,0.00,23.0,1.0
1,0.73,0.0,0.35,0.38,1.06,5.86,0.64,0.00,3.18,7.19,...,234.51,170.0,169.0,0.01,0.0,2.25,0.0,0.00,23.0,1.0
2,1.22,0.0,0.73,0.67,2.20,8.72,0.88,4.35,5.17,7.55,...,211.44,1431.0,1427.0,0.01,0.0,10.66,1.0,0.18,23.0,1.0
3,2.50,0.0,1.16,1.63,3.47,13.51,0.00,5.26,8.77,16.78,...,220.23,94.0,92.0,0.00,0.0,0.00,0.0,0.00,23.0,1.0
4,3.51,0.0,1.72,1.54,5.15,9.11,1.04,3.10,4.93,12.82,...,225.16,117.0,114.0,0.00,0.0,18.18,1.0,13.32,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.72,2.51,8.16,9.85,0.97,4.59,6.81,8.30,...,158.59,63.0,60.0,0.01,0.0,24.19,2.0,15.11,1.0,0.0
1036,1.76,0.0,0.00,0.69,2.43,7.57,0.74,2.72,0.00,11.93,...,143.80,47.0,44.0,0.01,0.0,46.05,1.0,25.79,1.0,0.0
1037,2.91,0.0,1.14,1.27,3.42,15.49,1.46,6.43,0.00,14.24,...,149.64,79.0,77.0,0.01,0.0,20.00,1.0,4.57,1.0,0.0
1038,3.79,0.0,1.79,1.48,5.36,15.70,1.37,6.37,11.09,18.82,...,451.37,75.0,70.0,0.01,0.0,0.00,3.0,19.58,1.0,0.0


In [12]:
euc_dist = {}

In [13]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)

Kolom : 1
[1.17, 1.21, 1.42, 2.27, 2.72, 3.09, 3.34]
min :  1.17
max :  3.34
mean :  2.174285714285714
fuzzy :  [(1.21, 0.03982930298719777), (1.42, 0.24893314366998584), (3.09, 0.21446078431372545), (2.72, 0.5318627450980389), (1.17, 0.0), (3.34, 0.0), (2.27, 0.9178921568627448)]
fuzzy :  None
pembilang :  4.594644206900398
penyebut :  1.9529781329316926
rata-rata tertimbang :  2.3526347425115284
row, col : 46 0
 
[0.0, 1.5, 1.54, 2.08, 2.75, 2.95, 3.84]
min :  0.0
max :  3.84
mean :  2.0942857142857143
fuzzy :  [(2.08, 0.9931787175989086), (0.0, 0.0), (2.75, 0.6243862520458265), (1.5, 0.7162346521145976), (1.54, 0.7353342428376535), (3.84, 0.0), (2.95, 0.5098199672667756)]
fuzzy :  None
pembilang :  7.493609541310624
penyebut :  3.578953831863762
rata-rata tertimbang :  2.093798884633357
row, col : 60 0
 
[0.28, 0.29, 0.29, 0.46, 0.56, 0.61, 1.76]
min :  0.28
max :  1.76
mean :  0.6071428571428571
fuzzy :  [(0.56, 0.8558951965065505), (0.61, 0.9975216852540271), (0.46, 0.550218340611

[0.0, 1.15, 1.26, 1.9, 1.96, 3.41, 10.44]
min :  0.0
max :  10.44
mean :  2.874285714285714
fuzzy :  [(0.0, 0.0), (1.15, 0.4000994035785288), (1.26, 0.4383697813121273), (10.44, 0.0), (3.41, 0.929191842900302), (1.96, 0.6819085487077535), (1.9, 0.6610337972166999)]
fuzzy :  None
pembilang :  6.7735093930375445
penyebut :  3.1106033737154113
rata-rata tertimbang :  2.17755482755329
row, col : 459 0
 
[0.0, 1.72, 2.86, 3.64, 3.85, 4.33, 4.38]
min :  0.0
max :  4.38
mean :  2.968571428571429
fuzzy :  [(0.0, 0.0), (4.38, 0.0), (4.33, 0.03542510121457478), (3.64, 0.5242914979757085), (2.86, 0.9634263715110682), (1.72, 0.5794032723772858), (3.85, 0.3755060728744939)]
fuzzy :  None
pembilang :  7.259483172468077
penyebut :  2.478052315953131
rata-rata tertimbang :  2.9295116675839297
row, col : 476 0
 
[0.0, 2.63, 3.73, 3.9, 4.35, 5.12, 7.46]
min :  0.0
max :  7.46
mean :  3.884285714285714
fuzzy :  [(0.0, 0.0), (7.46, 0.0), (3.9, 0.995605273671594), (2.63, 0.677087164398676), (5.12, 0.654414

[0.0, 0.0, 2.33, 2.63, 2.8, 4.58, 7.46]
min :  0.0
max :  7.46
mean :  2.8285714285714283
fuzzy :  [(0.0, 0.0), (2.8, 0.98989898989899), (0.0, 0.0), (2.63, 0.9297979797979798), (4.58, 0.621838371375694), (7.46, 0.0), (2.33, 0.8237373737373739)]
fuzzy :  None
pembilang :  9.984413680294619
penyebut :  3.365272714810038
rata-rata tertimbang :  2.966895858500495
row, col : 776 0
 
[0.0, 0.0, 1.26, 2.8, 2.98, 3.5, 5.95]
min :  0.0
max :  5.95
mean :  2.3557142857142854
fuzzy :  [(0.0, 0.0), (3.5, 0.681637519872814), (2.8, 0.8763910969793323), (2.98, 0.8263116057233704), (1.26, 0.534869617950273), (0.0, 0.0), (5.95, 0.0)]
fuzzy :  None
pembilang :  7.975970694769968
penyebut :  2.91920984052579
rata-rata tertimbang :  2.7322361633767946
row, col : 789 0
 
[0.0, 3.34, 5.47, 5.89, 6.24, 6.51, 7.63]
min :  0.0
max :  7.63
mean :  5.011428571428572
fuzzy :  [(0.0, 0.0), (7.63, 0.0), (5.89, 0.6644844517184945), (3.34, 0.6664766248574686), (5.47, 0.8248772504091655), (6.24, 0.530823786142935), (6

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 0 1
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 0 1
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 0 1
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang 

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 0 1
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 0 1
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 0 1
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang 

...

[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 912 27
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 913 27
 
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.42857142857142855
fuzzy :  [(0.0, 0.0), (1.0, 0.0), (1.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 914 27
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 941 27
 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.2857142857142857
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (0.0, 0.0), (1.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 942 27
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 943 27
 
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.2857142857142857
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (0.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  4.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 970 27
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.14285714285714285
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
row, col : 971 27
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 972 27
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 999 27
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 1000 27
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 1001 27
 
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.5714285714285714
fuzzy :  [(0.0, 1.0), (0.0, 1.0), (1.0, 1.0), 

[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  5.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 1028 27
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 1029 27
 
[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.8571428571428571
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0)]
fuzzy :  1.0
pembilang :  6.0
penyebut :  7.0
rata-rata tertimbang :  1.0
row, col : 1030 27
 
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]
median :  4
min :  0.0
max :  1.0
mean :  0.7142857142857143
fuzzy :  [(0.0, 1.0), (1.0, 1.0), (1.0, 1.0),

In [14]:
dummy

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
0,1.49,0.0,0.900000,0.811661,2.70,8.33,0.780000,4.520000,4.610000,6.800000,...,187.580000,160.0,159.0,0.01,0.0,14.990706,1.000000,10.090000,23.0,1.0
1,0.73,0.0,0.350000,0.380000,1.06,5.86,0.640000,5.184663,3.180000,7.190000,...,234.510000,170.0,169.0,0.01,0.0,2.250000,1.795181,18.337503,23.0,1.0
2,1.22,0.0,0.730000,0.670000,2.20,8.72,0.880000,4.350000,5.170000,7.550000,...,211.440000,1431.0,1427.0,0.01,0.0,10.660000,1.000000,0.180000,23.0,1.0
3,2.50,0.0,1.160000,1.630000,3.47,13.51,1.000983,5.260000,8.770000,16.780000,...,220.230000,94.0,92.0,0.01,0.0,18.005885,1.000000,0.180000,23.0,1.0
4,3.51,0.0,1.720000,1.540000,5.15,9.11,1.040000,3.100000,4.930000,12.820000,...,225.160000,117.0,114.0,0.01,0.0,18.180000,1.000000,13.320000,23.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1035,5.13,0.0,2.720000,2.510000,8.16,9.85,0.970000,4.590000,6.810000,8.300000,...,158.590000,63.0,60.0,0.01,0.0,24.190000,2.000000,15.110000,1.0,1.0
1036,1.76,0.0,1.064145,0.690000,2.43,7.57,0.740000,2.720000,5.521729,11.930000,...,143.800000,47.0,44.0,0.01,0.0,46.050000,1.000000,25.790000,1.0,1.0
1037,2.91,0.0,1.140000,1.270000,3.42,15.49,1.460000,6.430000,7.722851,14.240000,...,149.640000,79.0,77.0,0.01,0.0,20.000000,1.000000,4.570000,1.0,1.0
1038,3.79,0.0,1.790000,1.480000,5.36,15.70,1.370000,6.370000,11.090000,18.820000,...,451.370000,75.0,70.0,0.01,0.0,19.534982,3.000000,19.580000,1.0,1.0


In [15]:
dummy.describe()

,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),Shimmer (apq11),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,features 27,class
count,1040.000000,1040.0,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,1040.000000,...,1040.000000,1040.000000,1040.000000,1.040000e+03,1040.00000,1040.000000,1040.000000,1040.000000,1040.000000,1040.0
mean,2.647192,0.0,1.239330,1.334085,3.700477,12.850537,1.183762,5.667240,7.864378,12.075235,...,233.726658,109.140799,105.515753,1.000000e-02,0.00001,29.060150,1.623184,17.264418,12.822752,1.0
std,1.745998,0.0,0.969418,1.131398,2.902217,5.380419,0.412544,2.982521,4.750404,5.780988,...,121.003148,149.728651,149.000109,1.700847e-16,0.00031,19.357009,1.356602,12.984949,15.640296,0.0
min,0.190000,0.0,0.060000,0.080000,0.190000,1.190000,0.100000,0.500000,0.710000,0.520000,...,85.540000,5.000000,4.000000,1.000000e-02,0.00000,0.160000,1.000000,0.180000,1.000000,1.0
25%,1.510000,0.0,0.635763,0.670000,1.840000,9.367500,0.940000,3.730000,5.170000,8.267500,...,143.120000,42.000000,40.000000,1.000000e-02,0.00000,12.842500,1.000000,7.192569,1.000000,1.0
50%,2.350000,0.0,1.040000,1.106234,3.075000,12.196780,1.170000,5.120000,6.935000,11.282104,...,195.740000,64.000000,61.500000,1.000000e-02,0.00000,27.175000,1.000000,13.547811,5.000000,1.0
75%,3.350000,0.0,1.590000,1.660000,4.760000,15.397500,1.400000,6.843378,9.222500,14.889449,...,263.087500,112.000000,109.000000,1.000000e-02,0.00000,42.597500,2.000000,24.461072,23.000000,1.0
max,14.380000,0.0,8.020000,13.540000,24.050000,41.140000,2.720000,25.820000,72.860000,42.000000,...,597.970000,1490.000000,1489.000000,1.000000e-02,0.01000,88.160000,12.000000,69.120000,55.000000,1.0


# Klasifikasi

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [34]:
x = dummy.values
y = df.iloc[:, -1].values

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=15)

In [36]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [37]:
classifier = DecisionTreeClassifier()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[130   5]
 [  3 122]]
0.9692307692307692
